In [1]:
import time
import numpy as np
import yastn
import yastn.tn.mps as mps
import yastn.tn.fpeps as peps
from routines import NSpin12, MapHex127_r4, MapHex127_r3, gates_from_HH, gates_from_HH2, measure_H_ctm, measure_H_mps

In [2]:

fname = "/home/marek/hex/problems/transfer_Hamiltonian_ibm_kyiv_0_angles_16_0_p_20.pkl"
data = np.load(fname, allow_pickle=True)

In [10]:
# Hamiltonian
fname = "./problems/sherbrooke0v3.pkl"
data = np.load(fname, allow_pickle=True)
HC = data["H"]
HM = {(n,): 1 for n in range(127)}
#
angles = {"p=3": {"beta": (0.50502, 0.35713, 0.19264),
                  "gamma": (-0.14264, -0.26589, -0.34195)},
          "p=4": {"beta": (0.54321, 0.41806, 0.28615, 0.16041),
                  "gamma": (-0.12077, -0.22360, -0.29902, -0.35329)},
          "p=5": {"beta": (0.53822, 0.44776, 0.32923, 0.23056, 0.12587),
                  "gamma": (-0.11764, -0.19946, -0.268736, -0.321586, -0.34583)},}

In [11]:
# group spins
hex4 = MapHex127_r4()
ops = NSpin12(sym='dense')
#
# translate Hamiltonian to grouped sites
HHC4 = hex4.group_H(HC, ops.z, ops.I)
HHM4 = hex4.group_H(HM, ops.x, ops.I)

In [12]:
g1 = gates_from_HH(HHC4, 1)
g2 = gates_from_HH2(HHC4, 1)

In [16]:
len(g2.nn)

45

In [17]:
problem = "p=5"
D = 8  # PEPS bond dimension
#
betas = angles[problem]['beta']
gammas = angles[problem]['gamma']
#
# initial product state
#
geometry = peps.SquareLattice(dims=hex4.dims, boundary='obc')
vectors = {site: ops.vec_x((1,) * Nloc) for site, Nloc in hex4.s2N.items()}
psi4 = peps.product_peps(geometry, vectors)
#
#  run evolution
#
opts_svd_evol = {"D_total": D, "tol": 1e-10}
env4 = peps.EnvNTU(psi4, which='NN+')
#
infoss = []
t0 = time.time()
for step, (gamma, beta) in enumerate(zip(gammas, betas)):
    gates = gates_from_HH2(HHC4, gamma)
    infos = peps.evolution_step_(env4, gates, opts_svd=opts_svd_evol, symmetrize=False)
    infoss.append(infos)
    gates = gates_from_HH2(HHM4, beta)
    infos = peps.evolution_step_(env4, gates, opts_svd=opts_svd_evol, symmetrize=False)
    Delta = peps.accumulated_truncation_error(infoss, statistics='mean')
    print(f"Step: {step}; Evolution time: {time.time() - t0:3.2f}; Truncation error: {Delta:0.6f}")

Step: 0; Evolution time: 11.94; Truncation error: 0.000000
Step: 1; Evolution time: 24.00; Truncation error: 0.000000


KeyboardInterrupt: 

In [ ]:
opts_svd = {"D_total": D}
env4_ctm = peps.EnvCTM(psi4, init='eye')
t0 = time.time()
for info in env4_ctm.ctmrg_(max_sweeps=10, opts_svd=opts_svd, corner_tol=1e-5, iterator_step=1): #
    print("Ctmrg time:", time.time() - t0)
    print(info)

Ctmrg time: 4.272852897644043
CTMRG_out(sweeps=1, max_dsv=nan, converged=False, max_D=8)
Ctmrg time: 36.02344250679016
CTMRG_out(sweeps=2, max_dsv=0.6377591798931443, converged=False, max_D=8)
Ctmrg time: 65.38708257675171
CTMRG_out(sweeps=3, max_dsv=0.03364462425869813, converged=False, max_D=8)
Ctmrg time: 95.02489113807678
CTMRG_out(sweeps=4, max_dsv=0.0005063235051286282, converged=False, max_D=8)
Ctmrg time: 124.61877393722534
CTMRG_out(sweeps=5, max_dsv=3.257730211457399e-05, converged=False, max_D=8)
Ctmrg time: 154.97983598709106
CTMRG_out(sweeps=6, max_dsv=6.23994678587907e-07, converged=True, max_D=8)


In [36]:
env4_BP = peps.EnvBP(psi4)
t0 = time.time()
for info in env4_BP.iterate_(max_sweeps=10, iterator_step=1): #  corner_tol=1e-5,
    print("Ctmrg time:", time.time() - t0)
    print(info)

Ctmrg time: 0.45688509941101074
BP_out(sweeps=1, max_diff=np.float64(1.0121205834468923), converged=None)
Ctmrg time: 0.9782843589782715
BP_out(sweeps=2, max_diff=np.float64(0.39112019074261967), converged=None)
Ctmrg time: 1.4596245288848877
BP_out(sweeps=3, max_diff=np.float64(0.01628378160186533), converged=None)
Ctmrg time: 1.9591248035430908
BP_out(sweeps=4, max_diff=np.float64(0.0003889551523375502), converged=None)
Ctmrg time: 2.7113311290740967
BP_out(sweeps=5, max_diff=np.float64(7.2261843852377706e-06), converged=None)
Ctmrg time: 3.218902587890625
BP_out(sweeps=6, max_diff=np.float64(4.2721706174664415e-07), converged=None)
Ctmrg time: 3.682129383087158
BP_out(sweeps=7, max_diff=np.float64(3.469142316479831e-09), converged=None)
Ctmrg time: 4.117254734039307
BP_out(sweeps=8, max_diff=np.float64(2.7457865089129868e-11), converged=None)
Ctmrg time: 4.55954122543335
BP_out(sweeps=9, max_diff=np.float64(1.589288906375893e-13), converged=None)
Ctmrg time: 5.33596396446228
BP_out(

In [54]:
eng0, ZZ0 = measure_H_ctm(env4_ctm, HHC4)
print(f"Energy for {problem}: {eng0.real:0.4f}")

Energy for p=5: -145.6594


In [55]:
eng3, ZZ3 = measure_H_ctm(env4_BP, HHC4)
print(f"Energy for {problem}: {eng0.real:0.4f}")

Energy for p=5: -145.6594


In [10]:
env_mps = peps.EnvBoundaryMPS(psi4, opts_svd=opts_svd, setup='lrtb')
eng1, ZZ1 = measure_H_mps(env_mps, HHC4)
print(f"Energy for {problem}: {eng1.real:0.4f}")


Energy for p=3: -125.3721


In [12]:
max(abs(ZZ0[k] - ZZ1[k]) for k in ZZ0)

np.float64(3.5933787266406725e-05)

In [3]:
# group spins
hex3 = MapHex127_r3()
ops = NSpin12(sym='dense')
#
# translate Hamiltonian to grouped sites
HHC3 = hex3.group_H(HC, ops.z, ops.I)
HHM3 = hex3.group_H(HM, ops.x, ops.I)

In [4]:
problem = "p=3"
D = 8 # PEPS bond dimension
#
betas = angles[problem]['beta']
gammas = angles[problem]['gamma']
#
# initial product state
#
geometry = peps.SquareLattice(dims=hex3.dims, boundary='obc')
vectors = {site: ops.vec_x((1,) * Nloc) for site, Nloc in hex3.s2N.items()}
psi3 = peps.product_peps(geometry, vectors)


In [5]:
#
#  run evolution
#
opts_svd_evol = {"D_total": D, "tol": 1e-10}
env3 = peps.EnvBP(psi3, which='NN+BP')
#
infoss = []
t0 = time.time()
for step, (gamma, beta) in enumerate(zip(gammas, betas)):
    gates = gates_from_HH(HHC3, gamma)
    infos = peps.evolution_step_(env3, gates, opts_svd=opts_svd_evol, symmetrize=False)
    infoss.append(infos)
    gates = gates_from_HH(HHM3, beta)
    infos = peps.evolution_step_(env3, gates, opts_svd=opts_svd_evol, symmetrize=False)
    Delta = peps.accumulated_truncation_error(infoss, statistics='mean')
    print(f"Step: {step}; Evolution time: {time.time() - t0:3.2f}; Truncation error: {Delta:0.6f}")

Step: 0; Rvolution time: 2.42; Truncation error: 0.000000
Step: 1; Rvolution time: 6.34; Truncation error: 0.000000
Step: 2; Rvolution time: 26.50; Truncation error: 0.000000


In [ ]:
opts_svd = {"D_total":  2 * D}
env_mps = peps.EnvBoundaryMPS(psi3, opts_svd=opts_svd, setup='tlbr')

eng2, ZZ2 = measure_H_mps(env_mps, HHC3)
print(f"Energy for {problem}: {eng2:0.4f}")


In [7]:
eng2, ZZ2 = measure_H_mps(env_mps, HHC3)
print(f"Energy for {problem}: {eng2:0.4f}")

Energy for p=3: -125.3721


In [24]:
max(abs(ZZ0[k] - ZZ2[k]) for k in ZZ0)

np.float64(3.593378726568508e-05)